In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('sqlite_db_pythonsqlite.db')

In [3]:
curs = conn.cursor()

In [4]:
curs.execute("SELECT name FROM sqlite_master WHERE type = 'table'").fetchall()

[('Bookings',), ('Facilities',), ('Members',)]

In [5]:
curs.execute('SELECT * FROM Bookings').description

(('bookid', None, None, None, None, None, None),
 ('facid', None, None, None, None, None, None),
 ('memid', None, None, None, None, None, None),
 ('starttime', None, None, None, None, None, None),
 ('slots', None, None, None, None, None, None))

In [6]:
curs.execute('SELECT * FROM Facilities').description

(('facid', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('membercost', None, None, None, None, None, None),
 ('guestcost', None, None, None, None, None, None),
 ('initialoutlay', None, None, None, None, None, None),
 ('monthlymaintenance', None, None, None, None, None, None))

In [7]:
curs.execute('SELECT * FROM Members').description


(('memid', None, None, None, None, None, None),
 ('surname', None, None, None, None, None, None),
 ('firstname', None, None, None, None, None, None),
 ('address', None, None, None, None, None, None),
 ('zipcode', None, None, None, None, None, None),
 ('telephone', None, None, None, None, None, None),
 ('recommendedby', None, None, None, None, None, None),
 ('joindate', None, None, None, None, None, None))

Now we have all three tables, and the columns of each table identified

Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! 

In [8]:
q10 = curs.execute('SELECT name, revenue FROM(SELECT name, SUM(CASE WHEN memid = 0 THEN guestcost * slots ELSE membercost * slots END) AS revenue FROM Bookings INNER JOIN Facilities ON Bookings.facid = Facilities.facid GROUP BY name) AS inner_table WHERE revenue < 1000 ORDER BY revenue')

In [9]:
q10 = curs.fetchall()
print(q10)

[('Table Tennis', 180), ('Snooker Table', 240), ('Pool Table', 270)]


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [10]:
list11 = curs.execute('SELECT surname || ", " || firstname AS name, recommendedby FROM Members WHERE recommendedby > 0 ORDER BY name')

In [11]:
list11 = curs.fetchall()

In [12]:
q11df = pd.DataFrame(list11)
q11df

,0,1
0,"Bader, Florence",9
1,"Baker, Anne",9
2,"Baker, Timothy",13
3,"Boothe, Tim",3
4,"Butters, Gerald",1
5,"Coplin, Joan",16
6,"Crumpet, Erica",2
7,"Dare, Nancy",4
8,"Genting, Matthew",5
9,"Hunt, John",30


Q12: Find the facilities with their usage by member, but not guests

In [13]:
q12 = curs.execute('SELECT f.name, firstname|| " "|| surname as member_name, sum(slots), COUNT(f.name) FROM Facilities as f  INNER JOIN Bookings as b  ON f.facid = b.facid  INNER JOIN Members as m  ON b.memid = m.memid  WHERE m.memid > 0 GROUP BY f.name, member_name')

In [14]:
q12 = curs.fetchall()


In [15]:
q12df = pd.DataFrame(q12)

In [16]:
q12df.head(10)

,0,1,2,3
0,Badminton Court,Anna Mackenzie,96,30
1,Badminton Court,Anne Baker,30,10
2,Badminton Court,Burton Tracy,6,2
3,Badminton Court,Charles Owen,18,6
4,Badminton Court,Darren Smith,432,132
5,Badminton Court,David Jones,24,8
6,Badminton Court,David Pinker,21,7
7,Badminton Court,Douglas Jones,6,2
8,Badminton Court,Erica Crumpet,6,2
9,Badminton Court,Florence Bader,27,9


Q13: Find the facilities usage by month, but not guests

In [17]:
q13 = curs.execute('SELECT f.name, firstname|| " "|| surname as member_name, strftime("%m", starttime), sum(slots), COUNT(f.name) FROM Facilities as f  INNER JOIN Bookings as b  ON f.facid = b.facid  INNER JOIN Members as m  ON b.memid = m.memid  WHERE m.memid > 0 GROUP BY f.name, member_name, strftime("%m", starttime)')

In [18]:
q13 = curs.fetchall()


In [19]:
q13df = pd.DataFrame(q13)
q13df.head(10)

,0,1,2,3,4
0,Badminton Court,Anna Mackenzie,08,9,3
1,Badminton Court,Anna Mackenzie,09,87,27
2,Badminton Court,Anne Baker,08,3,1
3,Badminton Court,Anne Baker,09,27,9
4,Badminton Court,Burton Tracy,08,3,1
5,Badminton Court,Burton Tracy,09,3,1
6,Badminton Court,Charles Owen,08,12,4
7,Badminton Court,Charles Owen,09,6,2
8,Badminton Court,Darren Smith,07,123,38
9,Badminton Court,Darren Smith,08,180,55
